# Using Tools in LangChain


In [52]:
from langchain.memory import ConversationSummaryMemory
from langchain_openai import ChatOpenAI
from langchain.schema.messages import SystemMessage, HumanMessage
from dotenv import load_dotenv
import os

load_dotenv()

def create_basic_chain():
    chatModel = ChatOpenAI(temperature=0.7)
    memory = ConversationSummaryMemory(
        llm=chatModel,
        return_messages=True,
        memory_key="chat_history"
    )
    def processMessage(userQuestion: str):
        chat_history = memory.chat_memory.messages
        sysMessage = SystemMessage(content='''
        You are a helpful assistant! Your name is Bob.
        ''')
        humanMessage = HumanMessage(content=userQuestion)
        messages = [sysMessage] + chat_history + [humanMessage]
        response = chatModel.invoke(messages)
        memory.chat_memory.add_user_message(userQuestion)
        memory.chat_memory.add_ai_message(response.content)
        return response

    return processMessage

In [53]:
def test_basic_chat():
    chain = create_basic_chain()
    print("Basic Chat Test (type 'quit' to move to next section)")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            break
        try:
            response = chain(user_input)
            print("\nBob:", response.content, "\n")
        except Exception as e:
            print(f"\nError: {str(e)}\n")

test_basic_chat()

Basic Chat Test (type 'quit' to move to next section)

Bob: I'm sorry, I am not able to provide real-time stock market information. I recommend checking a financial news website or app for the most up-to-date information on stock market performance. If you have any other questions or need assistance with financial topics, feel free to ask! 



## 2. Adding Search Capability


In [55]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import Tool, initialize_agent, AgentType
from dotenv import load_dotenv
import os

load_dotenv()

def create_chain_with_search():
    chatModel = ChatOpenAI(temperature=0.7, verbose=True)
    
    # Initialize memory
    memory = ConversationSummaryMemory(
        llm=chatModel,
        return_messages=True,
        memory_key="chat_history"
    )
    
    #Step 1 : Initialize Tool
    search = DuckDuckGoSearchRun()
    
    
    tools = [
    # Step 2 : Create a Tool instance for the search tool
        Tool(
            name="Search",
            func=search.run,
            description="Useful for searching the internet for current information."
        ),
    ]
    
    # Step 3 : Initialize agent with memory
    agent = initialize_agent(
        tools=tools,
        llm=chatModel,
        agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
        memory=memory,
        verbose=True
    )
    
    return agent

def test_search_chat():
    agent = create_chain_with_search()
    print("Search-Enabled Chat Test (type 'quit' to move to next section)")
    
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            break
        try:
            response = agent.run(user_input)
            print("\nBob:", response, "\n")
        except Exception as e:
            print(f"\nError: {str(e)}\n")

if __name__ == "__main__":
    test_search_chat()

Search-Enabled Chat Test (type 'quit' to move to next section)


> Entering new AgentExecutor chain...
```json
{
    "action": "Search",
    "action_input": "stock market prices for today"
}
```
Observation: Today, the Dow Jones Industrial Average consists of the 30 most important market-leading companies on the American stock exchange and reflects their growth. (read more) Like the Swiss Market Index ... Currency traders talk near the screens showing the Korea Composite Stock Price Index (KOSPI), left, and the foreign exchange rate between U.S. dollar and South Korean won at a foreign exchange dealing room in Seoul, South Korea, Wednesday, Jan. 22, 2025. ... Stock market today: Wall Street pushes still higher on strong earnings, $500 billion AI ... Search for a stock to start your analysis. Accurate information on 100,000+ stocks and funds, including all the companies in the S&P500 index. See stock prices, news, financials, forecasts, charts and more. Explore the latest stock market t

## 3. Adding Weather Tool


In [57]:
import requests
from dotenv import load_dotenv
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI

load_dotenv()
api_key = "81a9e318874a398230f75adf8cd522a0"

# Step 1 - Define a tool to get weather data for a city
def get_weather(city_name: str) -> str:
    geocode_url = "http://api.openweathermap.org/geo/1.0/direct"
    geocode_params = {
        "q": city_name,
        "limit": 1,
        "appid": api_key
    }

    try:
        geocode_response = requests.get(geocode_url, params=geocode_params)
        geocode_data = geocode_response.json()

        if not geocode_data:
            return f"Error: Could not find coordinates for {city_name}."

        lat, lon = geocode_data[0]["lat"], geocode_data[0]["lon"]

        weather_url = "https://api.openweathermap.org/data/2.5/weather"
        weather_params = {
            "lat": lat,
            "lon": lon,
            "appid": api_key,
            "units": "metric",
            "lang": "en"
        }

        weather_response = requests.get(weather_url, params=weather_params)
        weather_data = weather_response.json()

        if weather_response.status_code == 200:
            return f"""
            Weather in {city_name}:
            Temperature: {weather_data['main']['temp']}°C
            Conditions: {weather_data['weather'][0]['description']}
            Humidity: {weather_data['main']['humidity']}%
            Wind Speed: {weather_data['wind']['speed']} m/s
            """
        else:
            return f"Error: {weather_data.get('message', 'Unknown error')} (Status Code: {weather_response.status_code})"

    except Exception as e:
        return f"Error: {str(e)}"

# Step 2 - Define a tool object for the get_weather function
weather_tool = Tool(
    name="get_weather",
    func=get_weather,
    description="Get current weather for a city. Input should be a city name."
)

llm = ChatOpenAI(temperature=0.7)

# Step 3 - Bind the weather tool to the LLM
llm_with_tools = llm.bind_tools([weather_tool])

def test_weather_tool():
    print("Weather Tool (type 'quit' to exit)")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            break

        response = llm_with_tools.invoke(user_input)

        if response.tool_calls:
            for tool_call in response.tool_calls:
                tool_name = tool_call["name"]
                tool_args = tool_call["args"]

                if tool_name == "get_weather":
                    city_name = tool_args.get("city_name", tool_args.get("__arg1"))
                    if not city_name:
                        print("Error: City name not found in tool arguments.")
                        continue

                    tool_result = get_weather(city_name)
                    print("\nTool Result:", tool_result)

                    conversational_response = llm_with_tools.invoke(
                        f"The user asked: '{user_input}'. Here is the weather data: {tool_result}. Provide a proper response."
                    )
                    print("\nAI Response:", conversational_response.content)
        else:
            print("\nAI Response:", response.content)

if __name__ == "__main__":
    test_weather_tool()

Weather Tool (type 'quit' to exit)

Tool Result: 
            Weather in New York:
            Temperature: -7.57°C
            Conditions: clear sky
            Humidity: 54%
            Wind Speed: 3.09 m/s
            

AI Response: Based on the current weather data in New York:
- Temperature: -7.57°C
- Conditions: clear sky
- Humidity: 54%
- Wind Speed: 3.09 m/s

It is unlikely to rain today in New York as the conditions are clear with no mention of precipitation.
